In [1]:
from amuse.lab import *
from amuse.units import (units, constants)
from amuse.couple import bridge
from amuse.units import quantities
from amuse.ext.orbital_elements import orbital_elements_from_binary
import matplotlib.pyplot as plt
import matplotlib.patches as patch
import numpy as np
from tqdm import tqdm
import sys
import csv
import timeit

# If you want to quickly check if it works: uncomment line 63 & 100 and comment line 62 & 99
# Also remember that if you ran it one time, that you delete the "mesa_gravity_hydro.amuse" file
# because otherwise you will get an error (because it cannot overwrite the file).

# Change parameter n and t_end (try n=5 and t_end=100yr again with new code for saving files)

# start run time
start = timeit.default_timer()

class friction:
    
    def __init__(self,hydro,particle):
        
        self.hydro = hydro
        self.particle = particle

    def get_gravity_at_point(self,eps,x,y,z):
        ##drag coefficient
        
        c_drag = 0.47
        
        result_ax = quantities.AdaptingVectorQuantity()
        result_ay = quantities.AdaptingVectorQuantity()
        result_az = quantities.AdaptingVectorQuantity()
        
        for i in range (1):
        
            hydro_density = self.hydro.get_hydro_state_at_point(self.particle[i].x,self.particle[i].y,self.particle[i].z,
                                                          self.particle[i].vx,self.particle[i].vy,self.particle[i].vz)[0]
            vx = self.particle[i].vx
            vy = self.particle[i].vy
            vz = self.particle[i].vz

            cross_section = np.pi*(self.particle[i].radius)**2

            ax = (0.5 * hydro_density * vx**2 * c_drag * cross_section)/self.particle[i].mass
            ay = (0.5 * hydro_density * vy**2 * c_drag * cross_section)/self.particle[i].mass
            az = (0.5 * hydro_density * vz**2 * c_drag * cross_section)/self.particle[i].mass
        
        
            result_ax.append(ax)
            result_ay.append(ay)
            result_az.append(az)
        
        return result_ax,result_ay,result_az

    
def main_nomoon(age,n):
    z = 0.0134
    RSun = 696340e5 #cm
    RRoche = 0.9*1.5e11 | units.m
    evolving_age = age | units.Myr
    n = n

    sun = Particles(1)
    sun.mass = 1 | units.MSun

    system = Particles(1)

    # Earth
    earth = system[0]
    earth.name = "Earth"
    earth.mass = units.MEarth(1)
    earth.radius = units.REarth(1) 
    earth.position = np.array((1,0.0,0.0)) | units.au
    earth.velocity = np.array((0.0,29780,0.0)) | units.ms

    converter_gravity = nbody_system.nbody_to_si(system.mass.sum(), earth.position.length())
    gravity = Huayno(converter_gravity)
    gravity.particles.add_particles(system)
    gravity.parameters.timestep = 0.1 | units.yr

    stellar = MESA()
    stellar.particles.add_particle(sun)
    stellar.parameters.metallicity = z

    evol_sun = stellar.particles[0]
    iterations = 0

    luminosity_sun = []
    temperature_sun = []

    while evol_sun.radius <= RRoche or (evol_sun.age > evolving_age and old_radius > evol_sun.radius):
    #while evol_sun.age <= evolving_age:
        # print(evol_sun.age.in_(units.Myr))

        luminosity_sun.append(evol_sun.luminosity.value_in(units.LSun))
        temperature_sun.append(evol_sun.temperature.value_in(units.K))

        stellar.evolve_model()

        old_radius = evol_sun.radius
        iterations += 1

        if iterations == 100:
            iterations = 0
            print("Radius sun =", evol_sun.radius.in_(units.RSun))
            print("Evolving age =", evol_sun.age.in_(units.Myr))    

    print("Evolving age =", evol_sun.age.in_(units.Myr))    

    # print current run time
    current_time = timeit.default_timer()
    print("Current run time =", int((current_time-start)/60), "min")

    target_core_mass = 0.8 * evol_sun.mass
    part_of_smaller_mass = 0.25
    N_h = (evol_sun.mass - target_core_mass) / (part_of_smaller_mass * (1 | units.MEarth))
    # N_h = (evol_sun.mass - target_core_mass) / (part_of_smaller_mass * moon.mass)

    sph_model = convert_stellar_model_to_SPH(evol_sun,
                                             round(N_h),
                                             with_core_particle = True,
                                             target_core_mass = 0.8 * evol_sun.mass,
                                             do_store_composition = False,
                                             base_grid_options=dict(type="fcc"))

    core = sph_model.core_particle.as_set()
    gas = sph_model.gas_particles

    print("Ngas =", len(gas), "Ncore =", core)
    print("RCore =", core.radius.value_in(units.m)[0])

    x_coordinates = dict()
    y_coordinates = dict()
    eccentricities = dict()
    semi_major_axis = dict()

    x_coordinates["core"] = []
    x_coordinates["gas_last"] = []
    x_coordinates["earth"] = []

    y_coordinates["core"] = []
    y_coordinates["gas_last"] = []
    y_coordinates["earth"] = []

    eccentricities["earth"] = []

    semi_major_axis["earth"] = []

    for i in range(n):

        converter_hydro = nbody_system.nbody_to_si(gas.mass.sum(), gas[-1].position.length())
        hydro = Fi(converter_hydro, mode="openmp")#, redirection="none")
        hydro.dm_particles.add_particle(core)
        hydro.gas_particles.add_particles(gas)
        # hydro.parameters.timestep = 1e-3 | units.yr

        gravity_hydro = bridge.Bridge()#use_threading=False)
        if hydro.get_hydro_state_at_point(gravity.particles[0].x, gravity.particles[0].y, gravity.particles[0].z,
                                          gravity.particles[0].vx, gravity.particles[0].vy, 
                                          gravity.particles[0].vz)[0].value_in(units.kg/(units.m**3)) == 0:
            gravity_hydro.add_system(gravity, (hydro,))
        else:
            gravity_hydro.add_system(gravity, (hydro, friction(hydro, gravity.particles)))
        gravity_hydro.timestep = 0.5 * gravity.parameters.timestep
        # gravity_hydro.timestep = 1e-3 | units.yr
        print("Gravity_hydro timestep =", gravity_hydro.timestep)

        t_end = 50 | units.yr
        model_time = 0 | units.yr
        time = np.arange(model_time.value_in(units.yr), t_end.value_in(units.yr), gravity_hydro.timestep.value_in(units.yr)) | units.yr
        for t in tqdm(time, desc="gravity_hydro"):
            x_coordinates["core"].append(core.x.value_in(units.cm)[0])
            x_coordinates["gas_last"].append(gas[-1].x.value_in(units.cm))
            x_coordinates["earth"].append(gravity.particles[0].x.value_in(units.cm))
        

            y_coordinates["core"].append(core.y.value_in(units.cm)[0])
            y_coordinates["gas_last"].append(gas[-1].y.value_in(units.cm))
            y_coordinates["earth"].append(gravity.particles[0].y.value_in(units.cm))
        

            ae = orbital_elements_from_binary([core, gravity.particles[0]], G=constants.G)[2:4]
            semi_major_axis["earth"].append(ae[0].value_in(units.m))
            eccentricities["earth"].append(ae[1])


            gravity_hydro.evolve_model(t)

        stellar.evolve_model(t_end)

        write_set_to_file(gravity_hydro.particles, "gravity_particles_t_end={}yr_n={}_i={}_test.amuse".format(t_end.value_in(units.yr), n, i), "amuse", append_to_file=False)
        write_set_to_file(hydro.particles, "hydro_particles_t_end={}yr_n={}_i={}_test.amuse".format(t_end.value_in(units.yr), n, i), "amuse", append_to_file=False)

        hydro.stop()

        sph_model = convert_stellar_model_to_SPH(evol_sun,
                                                round(N_h),
                                                with_core_particle = True,
                                                target_core_mass = 0.8 * evol_sun.mass,
                                                do_store_composition = False,
                                                base_grid_options=dict(type="fcc"))

        core = sph_model.core_particle.as_set()
        gas = sph_model.gas_particles

    data_file = open("nomoon_coordinates_t_end={}yr_n={}.csv".format(t_end.value_in(units.yr), n), mode='w')
    data_writer = csv.writer(data_file)
    for (key, x), y in zip(x_coordinates.items(), y_coordinates.values()):
        data_writer.writerow([key, x, y])
    data_file.close()

    data_file = open("nomoon_ae_t_end={}yr_n={}.csv".format(t_end.value_in(units.yr), n), mode='w')
    data_writer = csv.writer(data_file)
    for (key, a), e in zip(semi_major_axis.items(), eccentricities.values()):
        data_writer.writerow([key, a, e])
    data_file.close()



    stellar.stop()
    gravity.stop()

    # stop and print run time
    stop = timeit.default_timer()
    print("Time:", int((stop-start)/60), "min")

In [2]:
main_nomoon(12e3,15,'e1')

Radius sun = 1.30670041612 RSun
Evolving age = 9423.72289785 Myr
Radius sun = 1.8833254304 RSun
Evolving age = 11207.9450929 Myr
Radius sun = 2.83062891471 RSun
Evolving age = 11571.4038177 Myr
Radius sun = 4.18411669238 RSun
Evolving age = 11756.6002957 Myr
Radius sun = 5.61437204976 RSun
Evolving age = 11834.0552772 Myr
Radius sun = 7.05118647426 RSun
Evolving age = 11873.7675294 Myr
Radius sun = 8.84826772775 RSun
Evolving age = 11910.6653186 Myr
Radius sun = 12.6869495865 RSun
Evolving age = 11956.8263013 Myr
Radius sun = 17.5834846988 RSun
Evolving age = 11973.8296799 Myr
Radius sun = 22.3400031592 RSun
Evolving age = 11982.3831896 Myr
Radius sun = 26.9777835155 RSun
Evolving age = 11987.5085516 Myr
Radius sun = 31.5565132874 RSun
Evolving age = 11990.9481991 Myr
Radius sun = 36.0184425744 RSun
Evolving age = 11993.4009325 Myr
Radius sun = 40.4922089072 RSun
Evolving age = 11995.2822695 Myr
Radius sun = 44.9015892426 RSun
Evolving age = 11996.7508677 Myr
Radius sun = 49.3283752054

gravity_hydro: 100%|██████████████████████████| 300/300 [02:13<00:00,  2.25it/s]


Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|██████████████████████████| 300/300 [02:14<00:00,  2.23it/s]


Gravity_hydro timestep = 1577846.29968 s


...

Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|██████████████████████████| 300/300 [00:14<00:00, 20.03it/s]


Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|██████████████████████████| 300/300 [02:13<00:00,  2.24it/s]


Time: 24 min


In [2]:
main_nomoon(12e3,15,'e0')

Radius sun = 1.30670041612 RSun
Evolving age = 9423.72289785 Myr
Radius sun = 1.8833254304 RSun
Evolving age = 11207.9450929 Myr
Radius sun = 2.83062891471 RSun
Evolving age = 11571.4038177 Myr
Radius sun = 4.18411669238 RSun
Evolving age = 11756.6002957 Myr
Radius sun = 5.61437204976 RSun
Evolving age = 11834.0552772 Myr
Radius sun = 7.05118647426 RSun
Evolving age = 11873.7675294 Myr
Radius sun = 8.84826772775 RSun
Evolving age = 11910.6653186 Myr
Radius sun = 12.6869495865 RSun
Evolving age = 11956.8263013 Myr
Radius sun = 17.5834846988 RSun
Evolving age = 11973.8296799 Myr
Radius sun = 22.3400031592 RSun
Evolving age = 11982.3831896 Myr
Radius sun = 26.9777835155 RSun
Evolving age = 11987.5085516 Myr
Radius sun = 31.5565132874 RSun
Evolving age = 11990.9481991 Myr
Radius sun = 36.0184425744 RSun
Evolving age = 11993.4009325 Myr
Radius sun = 40.4922089072 RSun
Evolving age = 11995.2822695 Myr
Radius sun = 44.9015892426 RSun
Evolving age = 11996.7508677 Myr
Radius sun = 49.3283752054

gravity_hydro: 100%|██████████████████████████| 300/300 [01:36<00:00,  3.11it/s]


Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|██████████████████████████| 300/300 [01:34<00:00,  3.18it/s]


Gravity_hydro timestep = 1577846.29968 s


...

Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|██████████████████████████| 300/300 [01:42<00:00,  2.92it/s]


Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|██████████████████████████| 300/300 [01:38<00:00,  3.06it/s]


Time: 59 min


In [2]:
main_nomoon(12e3,10,'e0')

Radius sun = 1.30670041612 RSun
Evolving age = 9423.72289785 Myr
Radius sun = 1.8833254304 RSun
Evolving age = 11207.9450929 Myr
Radius sun = 2.83062891471 RSun
Evolving age = 11571.4038177 Myr
Radius sun = 4.18411669238 RSun
Evolving age = 11756.6002957 Myr
Radius sun = 5.61437204976 RSun
Evolving age = 11834.0552772 Myr
Radius sun = 7.05118647426 RSun
Evolving age = 11873.7675294 Myr
Radius sun = 8.84826772775 RSun
Evolving age = 11910.6653186 Myr
Radius sun = 12.6869495865 RSun
Evolving age = 11956.8263013 Myr
Radius sun = 17.5834846988 RSun
Evolving age = 11973.8296799 Myr
Radius sun = 22.3400031592 RSun
Evolving age = 11982.3831896 Myr
Radius sun = 26.9777835155 RSun
Evolving age = 11987.5085516 Myr
Radius sun = 31.5565132874 RSun
Evolving age = 11990.9481991 Myr
Radius sun = 36.0184425744 RSun
Evolving age = 11993.4009325 Myr
Radius sun = 40.4922089072 RSun
Evolving age = 11995.2822695 Myr
Radius sun = 44.9015892426 RSun
Evolving age = 11996.7508677 Myr
Radius sun = 49.3283752054

gravity_hydro: 100%|████████████████████████| 1000/1000 [05:40<00:00,  2.94it/s]


Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|████████████████████████| 1000/1000 [05:32<00:00,  3.01it/s]


Gravity_hydro timestep = 1577846.29968 s


...

Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|████████████████████████| 1000/1000 [05:25<00:00,  3.07it/s]


Gravity_hydro timestep = 1577846.29968 s


gravity_hydro: 100%|████████████████████████| 1000/1000 [05:19<00:00,  3.13it/s]


Time: 97 min
